## InfoSec Jupyterthon 2021 - Day 2

---
# Advanced Pandas

- Joins and merges [Ian, Ashwin]
- Exporting/Importing [Ian, Pete] 
- Using styles [Ian, Pete] 
- Reshaping/preprocessing data [Ashwin, Luis Francisco, Jose] 
- Time manipulation
- Other useful operations  [Ashwin, Luis Francisco, Ian] 


---

# Joins and merges [Ian, Ashwin] 


 ### Load some data and normalize it into:
 - Processes
 - ParentProcesses
 - Users

In [2]:
import pandas as pd

procs_df = pd.read_csv(
    "../data/process_tree.csv",
    parse_dates=["TimeCreatedUtc", "TimeGenerated"],
    index_col=0
)
parents = procs_df[["ProcessId", "ParentProcessName"]].drop_duplicates()
procs = (
    procs_df[["NewProcessId", "NewProcessName", "CommandLine", "ProcessId", "TimeCreatedUtc", "SubjectUserSid"]]
    .drop_duplicates()
    .rename(columns={"ProcessId": "ParentProcessId"})
)
users = procs_df[['SubjectUserSid', 'SubjectUserName', 'SubjectDomainName']].drop_duplicates()

print("original", len(procs_df))
print("procs", len(procs))
print("parents", len(parents))
print("users", len(users))

original 117
procs 117
parents 3
users 2


### Joining on Index using pd.concat

We saw using pd.concat to append rows in part 1

In [11]:
# Do some processing on the original DF
dec_logon_id = (
    pd.DataFrame(procs_df.SubjectLogonId.apply(lambda x: int(x, base=16)))
    .rename(columns={"SubjectLogonId": "SubjectLogonId_dec"})
)

dec_logon_id.head(5)

,SubjectLogonId_dec
0,16428071
1,16428071
2,16428071
3,16428071
4,16428071


#### pd.concat with `axis="columns"` or `axis=1` joins column-wise (horizontally)

In [17]:
(
    pd.concat([procs_df, dec_logon_id], axis="columns")
    .head()
    .filter(regex=".*Process.*|Sub.*")
)

,SubjectUserSid,SubjectUserName,SubjectDomainName,SubjectLogonId,NewProcessId,NewProcessName,ProcessId,ParentProcessName,SubjectLogonId_dec
0,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0xfaac27,0x1580,C:\Diagnostics\UserTmp\ftp.exe,0xbc8,C:\Windows\System32\cmd.exe,16428071
1,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0xfaac27,0x16fc,C:\Diagnostics\UserTmp\reg.exe,0xbc8,C:\Windows\System32\cmd.exe,16428071
2,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0xfaac27,0x1700,C:\Diagnostics\UserTmp\cmd.exe,0xbc8,C:\Windows\System32\cmd.exe,16428071
3,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0xfaac27,0x1728,C:\Diagnostics\UserTmp\rundll32.exe,0xbc8,C:\Windows\System32\cmd.exe,16428071
4,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1,0xfaac27,0x175c,C:\Diagnostics\UserTmp\rundll32.exe,0xbc8,C:\Windows\System32\cmd.exe,16428071


## Key-based Joins

Source tables

In [37]:
display(procs.head())
display(users)


,NewProcessId,NewProcessName,CommandLine,ParentProcessId,TimeCreatedUtc,SubjectUserSid
0,0x1580,C:\Diagnostics\UserTmp\ftp.exe,.\ftp -s:C:\RECYCLER\xxppyy.exe,0xbc8,2019-01-15 05:15:15.677,S-1-5-21-996632719-2361334927-4038480536-500
1,0x16fc,C:\Diagnostics\UserTmp\reg.exe,.\reg not /domain:everything that /sid:shines...,0xbc8,2019-01-15 05:15:16.167,S-1-5-21-996632719-2361334927-4038480536-500
2,0x1700,C:\Diagnostics\UserTmp\cmd.exe,"cmd /c ""systeminfo && systeminfo""",0xbc8,2019-01-15 05:15:16.277,S-1-5-21-996632719-2361334927-4038480536-500
3,0x1728,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32 /C 12345.exe,0xbc8,2019-01-15 05:15:16.340,S-1-5-21-996632719-2361334927-4038480536-500
4,0x175c,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32 /C c:\users\MSTICAdmin\12345.exe,0xbc8,2019-01-15 05:15:16.400,S-1-5-21-996632719-2361334927-4038480536-500


,SubjectUserSid,SubjectUserName,SubjectDomainName
0,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
115,S-1-5-18,MSTICAlertsWin1$,WORKGROUP


### Simple merge on common key

In [38]:
procs.merge(users, on="SubjectUserSid")

,NewProcessId,NewProcessName,CommandLine,ParentProcessId,TimeCreatedUtc,SubjectUserSid,SubjectUserName,SubjectDomainName
0,0x1580,C:\Diagnostics\UserTmp\ftp.exe,.\ftp -s:C:\RECYCLER\xxppyy.exe,0xbc8,2019-01-15 05:15:15.677,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
1,0x16fc,C:\Diagnostics\UserTmp\reg.exe,.\reg not /domain:everything that /sid:shines...,0xbc8,2019-01-15 05:15:16.167,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
2,0x1700,C:\Diagnostics\UserTmp\cmd.exe,"cmd /c ""systeminfo && systeminfo""",0xbc8,2019-01-15 05:15:16.277,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
3,0x1728,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32 /C 12345.exe,0xbc8,2019-01-15 05:15:16.340,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
4,0x175c,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32 /C c:\users\MSTICAdmin\12345.exe,0xbc8,2019-01-15 05:15:16.400,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
...,...,...,...,...,...,...,...,...
112,0x1434,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32.exe /C c:\windows\fonts\conhost.exe,0xbc8,2019-01-15 05:15:14.613,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
113,0x123c,C:\Diagnostics\UserTmp\regsvr32.exe,.\regsvr32 /u /s c:\windows\fonts\csrss.exe,0xbc8,2019-01-15 05:15:14.693,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
114,0x240,C:\Windows\System32\tasklist.exe,tasklist,0xbc8,2019-01-15 05:15:14.770,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1
115,0x15a0,C:\Windows\System32\win32calc.exe,"""C:\Windows\System32\win32calc.exe""",0x1580,2019-01-15 05:15:13.053,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAdmin,MSTICAlertsWin1


### Left joins (also right and outer)

In [39]:
procs.merge(users[1:], on="SubjectUserSid")

,NewProcessId,NewProcessName,CommandLine,ParentProcessId,TimeCreatedUtc,SubjectUserSid,SubjectUserName,SubjectDomainName
0,0xbc8,C:\Windows\System32\cmd.exe,cmd.exe /c c:\Diagnostics\WindowsSimulateDetec...,0x440,2019-01-15 05:15:03.047,S-1-5-18,MSTICAlertsWin1$,WORKGROUP


In [40]:
procs.merge(users[1:], on="SubjectUserSid", how="left")

,NewProcessId,NewProcessName,CommandLine,ParentProcessId,TimeCreatedUtc,SubjectUserSid,SubjectUserName,SubjectDomainName
0,0x1580,C:\Diagnostics\UserTmp\ftp.exe,.\ftp -s:C:\RECYCLER\xxppyy.exe,0xbc8,2019-01-15 05:15:15.677,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
1,0x16fc,C:\Diagnostics\UserTmp\reg.exe,.\reg not /domain:everything that /sid:shines...,0xbc8,2019-01-15 05:15:16.167,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
2,0x1700,C:\Diagnostics\UserTmp\cmd.exe,"cmd /c ""systeminfo && systeminfo""",0xbc8,2019-01-15 05:15:16.277,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
3,0x1728,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32 /C 12345.exe,0xbc8,2019-01-15 05:15:16.340,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
4,0x175c,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32 /C c:\users\MSTICAdmin\12345.exe,0xbc8,2019-01-15 05:15:16.400,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
...,...,...,...,...,...,...,...,...
112,0x1434,C:\Diagnostics\UserTmp\rundll32.exe,.\rundll32.exe /C c:\windows\fonts\conhost.exe,0xbc8,2019-01-15 05:15:14.613,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
113,0x123c,C:\Diagnostics\UserTmp\regsvr32.exe,.\regsvr32 /u /s c:\windows\fonts\csrss.exe,0xbc8,2019-01-15 05:15:14.693,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
114,0x240,C:\Windows\System32\tasklist.exe,tasklist,0xbc8,2019-01-15 05:15:14.770,S-1-5-21-996632719-2361334927-4038480536-500,NaN,NaN
115,0xbc8,C:\Windows\System32\cmd.exe,cmd.exe /c c:\Diagnostics\WindowsSimulateDetec...,0x440,2019-01-15 05:15:03.047,S-1-5-18,MSTICAlertsWin1$,WORKGROUP


### Joins where no common key

In [18]:
procs.merge(parents, on="ProcessId")

KeyError: 'ProcessId'

In [20]:
(
    procs.merge(parents, left_on="ParentProcessId", right_on="ProcessId")
    .head()
    .filter(regex=".*Process.*")
)

,NewProcessId,NewProcessName,ParentProcessId,ProcessId,ParentProcessName
0,0x1580,C:\Diagnostics\UserTmp\ftp.exe,0xbc8,0xbc8,C:\Windows\System32\cmd.exe
1,0x16fc,C:\Diagnostics\UserTmp\reg.exe,0xbc8,0xbc8,C:\Windows\System32\cmd.exe
2,0x1700,C:\Diagnostics\UserTmp\cmd.exe,0xbc8,0xbc8,C:\Windows\System32\cmd.exe
3,0x1728,C:\Diagnostics\UserTmp\rundll32.exe,0xbc8,0xbc8,C:\Windows\System32\cmd.exe
4,0x175c,C:\Diagnostics\UserTmp\rundll32.exe,0xbc8,0xbc8,C:\Windows\System32\cmd.exe


---

# Using Styles [Ian] 

- Max/min values 
- Value coloring 
- Inline bars 


In [4]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

# Generate a summary
summary_df = (
    net_df[["RemoteRegion", "TotalAllowedFlows", "L7Protocol"]]
    .groupby("RemoteRegion")
    .agg(
        FlowsSum = pd.NamedAgg("TotalAllowedFlows", "sum"),
        FlowsVar = pd.NamedAgg("TotalAllowedFlows", "var"),
        FlowsStdDev = pd.NamedAgg("TotalAllowedFlows", "std"),
        L7Prots = pd.NamedAgg("L7Protocol", "nunique"),
    )
)
summary_df

,FlowsSum,FlowsVar,FlowsStdDev,L7Prots
RemoteRegion,,,,
,814.0,57.267027,7.567498,6
canadacentral,5103.0,29.811223,5.459965,1
centralus,236.0,4.675897,2.162382,1
eastus,602.0,1.646154,1.283025,3
eastus2,1502.0,4.830914,2.197934,1
northeurope,82.0,0.492438,0.701739,1
southcentralus,817.0,8.882186,2.980300,1
westcentralus,59.0,0.017241,0.131306,1
westus,38.0,0.782609,0.884652,1


In [5]:
df_style = summary_df.style.highlight_max(color="blue").highlight_min(color="green")
df_style

,FlowsSum,FlowsVar,FlowsStdDev,L7Prots
RemoteRegion,,,,
,814.000000,57.267027,7.567498,6
canadacentral,5103.000000,29.811223,5.459965,1
centralus,236.000000,4.675897,2.162382,1
eastus,602.000000,1.646154,1.283025,3
eastus2,1502.000000,4.830914,2.197934,1
northeurope,82.000000,0.492438,0.701739,1
southcentralus,817.000000,8.882186,2.980300,1
westcentralus,59.000000,0.017241,0.131306,1
westus,38.000000,0.782609,0.884652,1


In [6]:
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)

summary_df.style.background_gradient(cmap=cm).format("{:.2f}")

,FlowsSum,FlowsVar,FlowsStdDev,L7Prots
RemoteRegion,,,,
,814.00,57.27,7.57,6.00
canadacentral,5103.00,29.81,5.46,1.00
centralus,236.00,4.68,2.16,1.00
eastus,602.00,1.65,1.28,3.00
eastus2,1502.00,4.83,2.20,1.00
northeurope,82.00,0.49,0.70,1.00
southcentralus,817.00,8.88,2.98,1.00
westcentralus,59.00,0.02,0.13,1.00
westus,38.00,0.78,0.88,1.00


In [7]:
summary_df.style.bar(color="blue").format("{:.2f}")

,FlowsSum,FlowsVar,FlowsStdDev,L7Prots
RemoteRegion,,,,
,814.00,57.27,7.57,6.00
canadacentral,5103.00,29.81,5.46,1.00
centralus,236.00,4.68,2.16,1.00
eastus,602.00,1.65,1.28,3.00
eastus2,1502.00,4.83,2.20,1.00
northeurope,82.00,0.49,0.70,1.00
southcentralus,817.00,8.88,2.98,1.00
westcentralus,59.00,0.02,0.13,1.00
westus,38.00,0.78,0.88,1.00


In [8]:
summary_df.style.set_properties(**{
    'background-color': 'black',
    'color': 'lawngreen',
    'font-family': 'consolas',
}).format("{:.2f}")


,FlowsSum,FlowsVar,FlowsStdDev,L7Prots
RemoteRegion,,,,
,814.00,57.27,7.57,6.00
canadacentral,5103.00,29.81,5.46,1.00
centralus,236.00,4.68,2.16,1.00
eastus,602.00,1.65,1.28,3.00
eastus2,1502.00,4.83,2.20,1.00
northeurope,82.00,0.49,0.70,1.00
southcentralus,817.00,8.88,2.98,1.00
westcentralus,59.00,0.02,0.13,1.00
westus,38.00,0.78,0.88,1.00


---

# Reshaping/preprocessing data?[Ashwin, Luis Francisco, Jose] 

- Dealing with nulls/NAs 
- Type conversion 
- Renaming columns
- Pandas operations: melt, explode, transpose, indexing/stack/unstack 
- Dealing with complex Python objects - explode 
- Tidy data - melt 


### Dealing with nulls/NAs

In [9]:
import pandas as pd

In [93]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

In [24]:
len(net_df)

1360

In [121]:
print(f"Null elements in DataFrame: {net_df.isnull().values.sum()} \n\
Rows with null elements: {net_df.shape[0] - net_df.dropna().shape[0]}")

Null elements in DataFrame: 24 
Rows with null elements: 8


In [12]:
net_df.isna().any()

TenantId               False
TimeGenerated          False
FlowStartTime          False
FlowEndTime            False
FlowIntervalEndTime    False
FlowType               False
ResourceGroup           True
VMName                  True
VMIPAddress            False
PublicIPs               True
SrcIP                  False
DestIP                 False
L4Protocol             False
L7Protocol             False
DestPort               False
FlowDirection          False
AllowedOutFlows        False
AllowedInFlows         False
DeniedInFlows          False
DeniedOutFlows         False
RemoteRegion           False
VMRegion               False
AllExtIPs              False
TotalAllowedFlows      False
dtype: bool

In [46]:
net_df.ResourceGroup.value_counts()

asihuntomsworkspacerg    1352
Name: ResourceGroup, dtype: int64

In [23]:
net_df[net_df["PublicIPs"].isna()]

,TenantId,TimeGenerated,FlowStartTime,FlowEndTime,FlowIntervalEndTime,FlowType,ResourceGroup,VMName,VMIPAddress,PublicIPs,...,DestPort,FlowDirection,AllowedOutFlows,AllowedInFlows,DeniedInFlows,DeniedOutFlows,RemoteRegion,VMRegion,AllExtIPs,TotalAllowedFlows
326,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-13 01:23:47.634,2019-02-13 00:55:10,2019-02-13 00:57:33,2019-02-13 01:00:00,IntraVNet,None,None,,None,...,445.0,O,3.0,0.0,0.0,0.0,,eastus,10.0.3.5,3.0
327,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-13 01:23:47.634,2019-02-13 00:55:10,2019-02-13 00:57:33,2019-02-13 01:00:00,IntraVNet,None,None,,None,...,445.0,I,0.0,3.0,0.0,0.0,,eastus,10.0.3.4,3.0
336,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 22:23:05.399,2019-02-12 21:55:43,2019-02-12 21:55:43,2019-02-12 22:00:00,IntraVNet,None,None,,None,...,22.0,O,1.0,0.0,0.0,0.0,,eastus,10.0.3.4,1.0
345,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 22:23:05.384,2019-02-12 21:54:04,2019-02-12 21:55:36,2019-02-12 22:00:00,IntraVNet,None,None,,None,...,22.0,O,6.0,0.0,0.0,0.0,,eastus,104.211.30.1,6.0
357,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 23:23:59.515,2019-02-12 22:22:35,2019-02-12 22:55:37,2019-02-12 23:00:00,IntraVNet,None,None,,None,...,22.0,O,12.0,0.0,0.0,0.0,,eastus,104.211.30.1,12.0
413,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 18:23:51.853,2019-02-12 17:26:19,2019-02-12 17:44:09,2019-02-12 18:00:00,IntraVNet,None,None,,None,...,445.0,O,6.0,0.0,0.0,0.0,,eastus,10.0.3.5,6.0
414,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 18:23:51.853,2019-02-12 17:26:19,2019-02-12 17:44:09,2019-02-12 18:00:00,IntraVNet,None,None,,None,...,445.0,I,0.0,6.0,0.0,0.0,,eastus,10.0.3.4,6.0
466,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-12 22:23:17.236,2019-02-12 21:55:43,2019-02-12 21:55:43,2019-02-12 22:00:00,IntraVNet,None,None,,None,...,22.0,I,0.0,1.0,0.0,0.0,,eastus,10.0.3.5,1.0


In [27]:
len(net_df.dropna())

1352

In [34]:
import numpy as np

In [52]:
net_df2 = net_df.fillna(value="N/A")

In [53]:
net_df2.ResourceGroup.value_counts()

asihuntomsworkspacerg    1352
N/A                         8
Name: ResourceGroup, dtype: int64

### Type Conversion

In [60]:
net_df.dtypes

TenantId                       object
TimeGenerated          datetime64[ns]
FlowStartTime          datetime64[ns]
FlowEndTime            datetime64[ns]
FlowIntervalEndTime    datetime64[ns]
FlowType                       object
ResourceGroup                  object
VMName                         object
VMIPAddress                    object
PublicIPs                      object
SrcIP                          object
DestIP                         object
L4Protocol                     object
L7Protocol                     object
DestPort                      float64
FlowDirection                  object
AllowedOutFlows               float64
AllowedInFlows                float64
DeniedInFlows                 float64
DeniedOutFlows                float64
RemoteRegion                   object
VMRegion                       object
AllExtIPs                      object
TotalAllowedFlows             float64
dtype: object

In [67]:
net_df.TotalAllowedFlows = net_df.TotalAllowedFlows.astype('str')

In [68]:
net_df.TotalAllowedFlows.dtypes

dtype('O')

In [69]:
net_df.TotalAllowedFlows = pd.to_numeric(net_df.TotalAllowedFlows)
#pd.to_datetime
#pd.to_timedelta

In [70]:
net_df.TotalAllowedFlows.dtypes

dtype('float64')

### Renaming columns

In [94]:
net_df.columns

Index(['TenantId', 'TimeGenerated', 'FlowStartTime', 'FlowEndTime',
       'FlowIntervalEndTime', 'FlowType', 'ResourceGroup', 'VMName',
       'VMIPAddress', 'PublicIPs', 'SrcIP', 'DestIP', 'L4Protocol',
       'L7Protocol', 'DestPort', 'FlowDirection', 'AllowedOutFlows',
       'AllowedInFlows', 'DeniedInFlows', 'DeniedOutFlows', 'RemoteRegion',
       'VMRegion', 'AllExtIPs', 'TotalAllowedFlows'],
      dtype='object')

In [95]:
net_df.rename(columns={"FlowStartTime": "FlowStartDateTime", "FlowEndTime": "FlowEndDateTime"}).columns

Index(['TenantId', 'TimeGenerated', 'FlowStartDateTime', 'FlowEndDateTime',
       'FlowIntervalEndTime', 'FlowType', 'ResourceGroup', 'VMName',
       'VMIPAddress', 'PublicIPs', 'SrcIP', 'DestIP', 'L4Protocol',
       'L7Protocol', 'DestPort', 'FlowDirection', 'AllowedOutFlows',
       'AllowedInFlows', 'DeniedInFlows', 'DeniedOutFlows', 'RemoteRegion',
       'VMRegion', 'AllExtIPs', 'TotalAllowedFlows'],
      dtype='object')

In [96]:
net_df.rename(str.lower, axis='columns').columns

Index(['tenantid', 'timegenerated', 'flowstarttime', 'flowendtime',
       'flowintervalendtime', 'flowtype', 'resourcegroup', 'vmname',
       'vmipaddress', 'publicips', 'srcip', 'destip', 'l4protocol',
       'l7protocol', 'destport', 'flowdirection', 'allowedoutflows',
       'allowedinflows', 'deniedinflows', 'deniedoutflows', 'remoteregion',
       'vmregion', 'allextips', 'totalallowedflows'],
      dtype='object')

In [97]:
net_df.columns

Index(['TenantId', 'TimeGenerated', 'FlowStartTime', 'FlowEndTime',
       'FlowIntervalEndTime', 'FlowType', 'ResourceGroup', 'VMName',
       'VMIPAddress', 'PublicIPs', 'SrcIP', 'DestIP', 'L4Protocol',
       'L7Protocol', 'DestPort', 'FlowDirection', 'AllowedOutFlows',
       'AllowedInFlows', 'DeniedInFlows', 'DeniedOutFlows', 'RemoteRegion',
       'VMRegion', 'AllExtIPs', 'TotalAllowedFlows'],
      dtype='object')

In [91]:
net_df.columns = ['tenantid', 'timegenerated', 'flowstartdatetime', 'flowenddatetime','flowintervalendtime', 'flowtype', 'resourcegroup', 'vmname','vmipaddress', 'publicips', 'srcip', 'destip', 'l4protocol','l7protocol', 'destport', 'flowdirection', 'allowedoutflows','allowedinflows', 'deniedinflows', 'deniedoutflows', 'remoteregion','vmregion', 'allextips', 'totalallowedflows']

### Pandas operations: melt, explode, transpose, indexing/stack/unstack

### Dealing with complex Python objects - explode

In [116]:
net_df.PublicIPs.head(10)

0                               [13.67.143.117]
1                                [40.77.232.95]
2                  [13.65.107.32, 40.124.45.19]
3                  [13.65.107.32, 40.124.45.19]
4                                [20.38.98.100]
5                               [13.67.143.117]
6                [13.71.172.128, 13.71.172.130]
7                [13.71.172.128, 13.71.172.130]
8    [65.55.44.109, 40.77.228.69, 65.55.44.108]
9    [65.55.44.109, 40.77.228.69, 65.55.44.108]
Name: PublicIPs, dtype: object

In [103]:
net_df.PublicIPs.count()

1352

In [105]:
net_df_ext = net_df.explode("PublicIPs")

In [114]:
net_df_ext.PublicIPs.head(10)

0    13.67.143.117
1     40.77.232.95
2     13.65.107.32
2     40.124.45.19
3     13.65.107.32
3     40.124.45.19
4     20.38.98.100
5    13.67.143.117
6    13.71.172.128
6    13.71.172.130
Name: PublicIPs, dtype: object

In [111]:
len(net_df_ext.PublicIPs.unique())

123

### Tidy data - melt
Pandas.melt() unpivots a DataFrame from wide format to long format.  
melt() function is useful to message a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are unpivoted to the row axis, leaving just two non-identifier columns, variable and value. 

In [150]:
net_df_min = net_df[["FlowType", "AllExtIPs", "TotalAllowedFlows"]]

In [152]:
pd.melt(net_df_min, 
         id_vars=['AllExtIPs'])

,AllExtIPs,variable,value
0,13.67.143.117,FlowType,AzurePublic
1,40.77.232.95,FlowType,AzurePublic
2,13.65.107.32,FlowType,AzurePublic
3,40.124.45.19,FlowType,AzurePublic
4,20.38.98.100,FlowType,AzurePublic
...,...,...,...
2715,13.71.172.130,TotalAllowedFlows,23.0
2716,40.77.232.95,TotalAllowedFlows,1.0
2717,52.168.138.145,TotalAllowedFlows,4.0
2718,23.215.98.90,TotalAllowedFlows,2.0


### Traspose

### Indexing, Stack and Unstack

In [ ]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

In [191]:
net_df_agg = net_df.groupby("AllExtIPs").agg({"TotalAllowedFlows":['mean', 'min', 'max'],
                                              "AllowedOutFlows":['mean', 'min', 'max'],
                                              "AllowedInFlows":['mean', 'min', 'max']})

In [192]:
net_df_agg.head()

TotalAllowedFlows            AllowedOutFlows             \
                           mean  min   max            mean  min   max   
AllExtIPs                                                               
10.0.3.4               3.333333  1.0   6.0        0.333333  0.0   1.0   
10.0.3.5               3.333333  1.0   6.0        3.000000  0.0   6.0   
104.211.30.1           9.000000  6.0  12.0        9.000000  6.0  12.0   
104.40.17.153          1.750000  1.0   2.0        1.750000  1.0   2.0   
104.43.212.12          2.166667  1.0   4.0        2.166667  1.0   4.0   

              AllowedInFlows            
                        mean  min  max  
AllExtIPs                               
10.0.3.4            3.000000  0.0  6.0  
10.0.3.5            0.333333  0.0  1.0  
104.211.30.1        0.000000  0.0  0.0  
104.40.17.153       0.000000  0.0  0.0  
104.43.212.12       0.000000  0.0  0.0

In [205]:
net_df_agg["TotalAllowedFlows"]["mean"]

AllExtIPs
10.0.3.4          3.333333
10.0.3.5          3.333333
104.211.30.1      9.000000
104.40.17.153     1.750000
104.43.212.12     2.166667
                   ...    
90.130.70.73      1.000000
99.84.104.63      7.000000
99.84.106.178    10.000000
99.84.106.27     10.000000
99.84.106.92     10.000000
Name: mean, Length: 125, dtype: float64

In [200]:
idx = pd.IndexSlice
net_df_agg.loc[:,idx[:,'mean']]

,TotalAllowedFlows,AllowedOutFlows,AllowedInFlows
,mean,mean,mean
AllExtIPs,,,
10.0.3.4,3.333333,0.333333,3.000000
10.0.3.5,3.333333,3.000000,0.333333
104.211.30.1,9.000000,9.000000,0.000000
104.40.17.153,1.750000,1.750000,0.000000
104.43.212.12,2.166667,2.166667,0.000000
...,...,...,...
90.130.70.73,1.000000,1.000000,0.000000
99.84.104.63,7.000000,7.000000,0.000000


In [209]:
net_df_agg_stacked = net_df_agg.stack()

In [215]:
net_df_agg_stacked.head()

TotalAllowedFlows  AllowedOutFlows  AllowedInFlows
AllExtIPs                                                         
10.0.3.4  mean           3.333333         0.333333        3.000000
          min            1.000000         0.000000        0.000000
          max            6.000000         1.000000        6.000000
10.0.3.5  mean           3.333333         3.000000        0.333333
          min            1.000000         0.000000        0.000000

In [221]:
net_df_agg_stacked.loc[("10.0.3.4","mean"),"TotalAllowedFlows"]

3.3333333333333335

In [219]:
net_df_agg_stacked.unstack().head()

TotalAllowedFlows            AllowedOutFlows             \
                           mean  min   max            mean  min   max   
AllExtIPs                                                               
10.0.3.4               3.333333  1.0   6.0        0.333333  0.0   1.0   
10.0.3.5               3.333333  1.0   6.0        3.000000  0.0   6.0   
104.211.30.1           9.000000  6.0  12.0        9.000000  6.0  12.0   
104.40.17.153          1.750000  1.0   2.0        1.750000  1.0   2.0   
104.43.212.12          2.166667  1.0   4.0        2.166667  1.0   4.0   

              AllowedInFlows            
                        mean  min  max  
AllExtIPs                               
10.0.3.4            3.000000  0.0  6.0  
10.0.3.5            0.333333  0.0  1.0  
104.211.30.1        0.000000  0.0  0.0  
104.40.17.153       0.000000  0.0  0.0  
104.43.212.12       0.000000  0.0  0.0

---
# Pivoting/pivot tables [Ashwin]


---
# Time manipulation [Ashwin] 

- Timezone considerations 
- Grouping by time 
- Resample.... 


---
# Other Useful operations  [Ashwin, Luis Francisco, Ian] 

- Chaining multiple operations with "." 
- Including external functions with pipe 
- Apply, assign, others ???? 

### Chaining multiple operations with "."

In [260]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

In [233]:
net_df[["TenantId","AllExtIPs"]].groupby("AllExtIPs").agg("count")

,TenantId
AllExtIPs,
10.0.3.4,3
10.0.3.5,3
104.211.30.1,2
104.40.17.153,4
104.43.212.12,12
...,...
90.130.70.73,2
99.84.104.63,1
99.84.106.178,1


In [236]:
net_df[["TenantId","AllExtIPs"]].groupby("AllExtIPs").agg("count").sort_values(by="TenantId", ascending=False)

,TenantId
AllExtIPs,
65.55.44.109,139
13.71.172.130,136
52.168.138.145,117
40.124.45.19,115
13.71.172.128,114
...,...
23.45.181.178,1
23.45.181.176,1
23.45.181.160,1


In [237]:
net_df[["TenantId","AllExtIPs"]].groupby("AllExtIPs").agg("count").sort_values(by="TenantId", ascending=False).head(5)

,TenantId
AllExtIPs,
65.55.44.109,139
13.71.172.130,136
52.168.138.145,117
40.124.45.19,115
13.71.172.128,114


In [243]:
net_df[["TenantId","AllExtIPs"]].groupby("AllExtIPs").agg("count").sort_values(by="TenantId", ascending=False).head(5).index.to_list()

['65.55.44.109',
 '13.71.172.130',
 '52.168.138.145',
 '40.124.45.19',
 '13.71.172.128']

### External functions with pipe

In [248]:
def fill_missing_values(df):
    for col in df.columns:
        df[col].fillna("N/A", inplace=True)
    return df

def drop_duplicates(df, column_name):
    df = df.drop_duplicates(subset=column_name)
    return df

In [245]:
net_df.value_counts("AllExtIPs")

AllExtIPs
65.55.44.109      139
13.71.172.130     136
52.168.138.145    117
40.124.45.19      115
13.71.172.128     114
                 ... 
23.45.181.178       1
23.45.181.176       1
23.45.181.160       1
23.45.180.34        1
99.84.106.92        1
Length: 125, dtype: int64

In [256]:
net_df.isnull().values.sum()

24

In [253]:
net_df.pipe(fill_missing_values).isnull().values.sum()

0

In [258]:
len(net_df)

1360

In [259]:
net_df.pipe(drop_duplicates, "AllExtIPs").count()

TenantId               125
TimeGenerated          125
FlowStartTime          125
FlowEndTime            125
FlowIntervalEndTime    125
FlowType               125
ResourceGroup          122
VMName                 122
VMIPAddress            125
PublicIPs              122
SrcIP                  125
DestIP                 125
L4Protocol             125
L7Protocol             125
DestPort               125
FlowDirection          125
AllowedOutFlows        125
AllowedInFlows         125
DeniedInFlows          125
DeniedOutFlows         125
RemoteRegion           125
VMRegion               125
AllExtIPs              125
TotalAllowedFlows      125
dtype: int64

In [319]:
net_df = pd.read_pickle("../data/az_net_comms_df.pkl")

In [268]:
len(net_df)

1360

In [264]:
net_df_pipe = (net_df.pipe(drop_duplicates, "AllExtIPs").pipe(fill_missing_values))

In [266]:
net_df_pipe.isnull().values.sum()

0

### External functions with apply

In [269]:
import pandas as pd
import numpy as np

def adder(element1, element2):
    return element1+element2


df = pd.DataFrame(np.random.randn(10, 3), columns=['Col1', 'Col2', 'Col3'])
print(df)

df1= df.apply(adder, args=([10]))
#df1= df.apply(adder, element2=10)

print(df1)

       Col1      Col2      Col3
0  0.379748 -0.459915 -1.630365
1  0.733204 -0.477975  0.297341
2 -1.187185  0.367660 -1.602276
3  0.500678  0.226610 -0.431865
4 -0.199604  0.131271 -0.173194
5 -0.063522  0.687468  0.009924
6 -1.137589  1.686923 -0.835308
7 -0.120839 -0.893021 -0.423119
8  0.320839  1.275479 -0.092932
9  1.274244 -0.363499 -1.022512
        Col1       Col2       Col3
0  10.379748   9.540085   8.369635
1  10.733204   9.522025  10.297341
2   8.812815  10.367660   8.397724
3  10.500678  10.226610   9.568135
4   9.800396  10.131271   9.826806
5   9.936478  10.687468  10.009924
6   8.862411  11.686923   9.164692
7   9.879161   9.106979   9.576881
8  10.320839  11.275479   9.907068
9  11.274244   9.636501   8.977488


In [306]:
net_df.head()

,TenantId,TimeGenerated,FlowStartTime,FlowEndTime,FlowIntervalEndTime,FlowType,ResourceGroup,VMName,VMIPAddress,PublicIPs,SrcIP,DestIP,L4Protocol,L7Protocol,DestPort,FlowDirection,AllowedOutFlows,AllowedInFlows,DeniedInFlows,DeniedOutFlows,RemoteRegion,VMRegion,AllExtIPs,TotalAllowedFlows
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 13:23:59.512,2019-02-14 12:21:58,2019-02-14 12:21:58,2019-02-14 13:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[13.67.143.117],,,T,https,443.0,O,1.0,0.0,0.0,0.0,centralus,eastus,13.67.143.117,1.0
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 13:23:59.512,2019-02-14 12:29:02,2019-02-14 12:29:02,2019-02-14 13:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[40.77.232.95],,,T,https,443.0,O,1.0,0.0,0.0,0.0,westcentralus,eastus,40.77.232.95,1.0
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 03:26:06.765,2019-02-14 02:08:46,2019-02-14 02:48:45,2019-02-14 03:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,"[13.65.107.32, 40.124.45.19]",,,T,https,443.0,O,4.0,0.0,0.0,0.0,southcentralus,eastus,13.65.107.32,4.0
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 03:26:06.765,2019-02-14 02:08:46,2019-02-14 02:48:45,2019-02-14 03:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,"[13.65.107.32, 40.124.45.19]",,,T,https,443.0,O,4.0,0.0,0.0,0.0,southcentralus,eastus,40.124.45.19,4.0
4,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 03:26:06.828,2019-02-14 02:30:56,2019-02-14 02:30:56,2019-02-14 03:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,[20.38.98.100],,,T,https,443.0,O,1.0,0.0,0.0,0.0,eastus,eastus,20.38.98.100,1.0


In [320]:
net_df_ext = net_df.explode("PublicIPs")

In [314]:
def get_country(ip):
    try:
        x = reader.country(ip)
    except:
        return np.nan
    try:
        return x.country.name if x else np.nan
    except KeyError:
        return np.nan

In [ ]:
!pip install geoip2

In [315]:
import geoip2.database
with geoip2.database.Reader(r'./Data/GeoLite2-Country.mmdb') as reader:
    unique_ips = net_df_ext['PublicIPs'].unique()
    unique_ips = pd.Series(unique_ips, index = unique_ips)
    net_df_ext['PublicIPCountry'] = net_df_ext['PublicIPs'].map(unique_ips.apply(get_country))

In [317]:
net_df_ext.value_counts("PublicIPCountry")

PublicIPCountry
United States     1885
Canada             515
Ireland             66
United Kingdom       4
Sweden               2
dtype: int64

In [318]:
net_df_ext.head()

,TenantId,TimeGenerated,FlowStartTime,FlowEndTime,FlowIntervalEndTime,FlowType,ResourceGroup,VMName,VMIPAddress,PublicIPs,SrcIP,DestIP,L4Protocol,L7Protocol,DestPort,FlowDirection,AllowedOutFlows,AllowedInFlows,DeniedInFlows,DeniedOutFlows,RemoteRegion,VMRegion,AllExtIPs,TotalAllowedFlows,PublicIPCountry
0,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 13:23:59.512,2019-02-14 12:21:58,2019-02-14 12:21:58,2019-02-14 13:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,13.67.143.117,,,T,https,443.0,O,1.0,0.0,0.0,0.0,centralus,eastus,13.67.143.117,1.0,United States
1,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 13:23:59.512,2019-02-14 12:29:02,2019-02-14 12:29:02,2019-02-14 13:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,40.77.232.95,,,T,https,443.0,O,1.0,0.0,0.0,0.0,westcentralus,eastus,40.77.232.95,1.0,United States
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 03:26:06.765,2019-02-14 02:08:46,2019-02-14 02:48:45,2019-02-14 03:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,13.65.107.32,,,T,https,443.0,O,4.0,0.0,0.0,0.0,southcentralus,eastus,13.65.107.32,4.0,United States
2,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 03:26:06.765,2019-02-14 02:08:46,2019-02-14 02:48:45,2019-02-14 03:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,40.124.45.19,,,T,https,443.0,O,4.0,0.0,0.0,0.0,southcentralus,eastus,13.65.107.32,4.0,United States
3,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-14 03:26:06.765,2019-02-14 02:08:46,2019-02-14 02:48:45,2019-02-14 03:00:00,AzurePublic,asihuntomsworkspacerg,msticalertswin1,10.0.3.5,13.65.107.32,,,T,https,443.0,O,4.0,0.0,0.0,0.0,southcentralus,eastus,40.124.45.19,4.0,United States
